In [1]:
import numpy as np
import re
import os
import csv
import math
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.model_selection import cross_val_score

In [2]:
def get_file_len(filename):
    length = 0
    with open(filename, 'r', encoding='ISO-8859-1') as f:
        content = f.readlines()
        for i in content:
            length = length + len(i)
        f.close()
    return length

In [3]:
def get_file_len(filename):
    length = 0
    with open(filename, 'r', encoding='ISO-8859-1') as f:
        content = f.readlines()
        for i in content:
            length = length + len(i)
        f.close()
    return length

In [4]:
def calc_ent(filename):
    file = ""

    with open(filename, 'r', encoding='ISO-8859-1') as f:
        lines = f.readlines()
        for i in lines:
            file = file + i

    num_entries = len(file)
    # 计算分类标签label出现的次数
    label_counts = {}
    # the number of unique elements and their occurance
    for featVec in file:
        # 将当前实例的标签存储，即每一行数据的最后一个数据代表的是标签
        current_label = featVec[-1]
        # 为所有可能的分类创建字典，如果当前的健值不存在，则扩展字典并将当前健值加入
        if current_label not in label_counts.keys():
            label_counts[current_label] = 0
            label_counts[current_label] += 1
    # 对于label标签的占比，求出label标签的香农墒
    shannon_ent = 0.0
    for key in label_counts:
        # 使所有类标签的发生频率计算类别出现的概率
        prob = float(label_counts[key]) / num_entries
        shannon_ent -= prob * math.log(prob, 2)
    return shannon_ent

In [5]:
def max_word(filename):
    content = ""
    with open(filename, 'r', encoding='ISO-8859-1') as f:
        c = f.readlines()
        for i in c:
            content = content + i

    f.close()
    text = content.split()
    maxone = 0
    for i in text:
        if len(i) > maxone:
            maxone = len(i)

    return (maxone)


In [6]:
def get_file_func(filename):
    content = ''
    content_list = []
    with open(filename, 'r', encoding='ISO-8859-1') as f:
        c = f.readlines()
        for i in c:
            content = content + i.strip('\n')
        f.close()
    content_list = re.split(r'\(|\)|\[|\]|\{|\}|\s|\.', content)
    max_length = 0
    for i in content_list:
        if len(i) > max_length:
            max_length = len(i)
        else:
            pass
    # print(content_list)
    count_exec = 0
    count_file = 0
    count_zip = 0
    count_code = 0
    count_chr = 0
    count_re = 0
    count_other = 0
    for i in content_list:
        if 'assert' in i or 'system' in i or 'eval' in i or 'cmd_shell' in i or 'shell_exec' in i:
            count_exec = count_exec + 1
        if 'file_get_contents' in i or 'fopen' in i or 'fwrite' in i or 'readdir' in i or 'scandir' in i or 'opendir' in i or 'curl' in i:
            count_file = count_file + 1
        if 'base64_encode' in i or 'base64_decode' in i:
            count_code = count_code + 1
        if 'gzcompress' in i or 'gzuncompress' in i or 'gzinflate' in i or 'gzdecode' in i:
            count_zip = count_zip + 1
        if 'chr' in i or 'ord' in i:
            count_chr + count_chr + 1
        if 'str_replace' in i or 'preg_replace' in i or 'substr' in i:
            count_re = count_re + 1
        if 'create_function' in i or 'pack' in i:
            count_other = count_other + 1
    # print(x)
    return (max_length, count_exec, count_file, count_zip, count_code, count_chr, count_re, count_other)



In [7]:
def get_files(dir, suffix):  # 查找根目录，文件后缀
    res = []
    for root, directory, files in os.walk(dir):  # =>当前根,根下目录,目录下的文件
        for filename in files:
            name, suf = os.path.splitext(filename)  # =>文件名,文件后缀
            if suf == suffix:
                res.append(os.path.join(root, filename))  # =>吧一串字符串组合成路径
    return res

In [8]:
def get_features(temp, value):
    temp.append(calc_ent(value))
    temp.append(get_file_len(value))
    temp.append(max_word(value))
    file_func = get_file_func(value)
    for index in file_func:
        temp.append(index)
    return temp

In [9]:
white_file_name = []
black_file_name = []
white_file = []
black_file = []
white_label = []
black_label = []

for file in get_files("./white", '.php'):  # =>查找以.php结尾的文件
    white_file_name.append(file)
    white_label.append(0)

for file in get_files("./black", '.php'):  # =>查找以.php结尾的文件
    black_file_name.append(file)
    black_label.append(1)

csvFile = open("csvData.csv", "w", newline='')  # 创建csv文件
writer = csv.writer(csvFile, delimiter=",")
header = ["data", "label"]  # 写入列的名称
# 写入多行用writerows                                #写入多行
writer.writerow(header)
writer.writerows(zip(white_file_name, white_label))
writer.writerows(zip(black_file_name, black_label))
csvFile.close()

print("读取名字完成")

name = []
label = []
temp = []
data = []
LaBel = []

with open('csvData.csv', 'rb') as f:
    data_value = [x.decode('utf-8').strip() for x in f.readlines()]
    for i in data_value:
        if i[-1:] == '0' or i[-1:] == '1':
            name.append(i[:-2])
            label.append(int(i[-1:]))
    f.close()

for i in range(0, len(name)):
    # for i in range(0,250):
    if len(name[i]) != 0:
        temp = get_features(temp, name[i])
        data.append(temp)
        LaBel.append(label[i])
        temp = []

# test

data = np.array(data)
LaBel = np.array(LaBel)

X_train, X_test, y_train, y_test = train_test_split(data, LaBel, test_size=0.3, random_state=1)
sc = StandardScaler()
sc.fit(X_train)
# 对数据集进行处理

print("***************start****************")

clf = XGBClassifier()
clf_save = clf.fit(X_train, y_train)
test_score = clf.score(X_test, y_test)

print("**************saveing***************")

score = cross_val_score(clf, data, LaBel, cv=5)
print(score)

print("Training score:%f" % (clf.score(X_train, y_train)))
print("Testing score:%f" % (clf.score(X_test, y_test)))

y_pred = clf.predict(X_test)

print("Testing accuracy score:%f" % (accuracy_score(y_test, y_pred)))
print("Testing precision score:%f" % (precision_score(y_test, y_pred)))
print("Testing recall score:%f" % (recall_score(y_test, y_pred)))




读取名字完成
***************start****************
**************saveing***************
[ 0.97769517  0.96468401  0.97951583  0.93283582  0.84701493]
Training score:0.988824
Testing score:0.960298
Testing accuracy score:0.960298
Testing precision score:0.961694
Testing recall score:0.973469
